### Correlation between news and stock Price

In [12]:
# import all necssary libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')
sys.path.insert(0, os.path.dirname(os.getcwd()))

from src.correlation_analysis import *

##### Load and merge the dataset

In [13]:
# get data path
folder_path = '../data/yfinance_data/'
# Automatically merge the data and assign to stock_analyzer
corr_analyser =CorrelationAnalyzer(folder_path=folder_path)
# Access the merged data
print(corr_analyser.data.head())
# save the merged data to a CSV file

         Date      Open      High       Low     Close  Adj Close      Volume  \
0  1997-05-15  0.121875  0.125000  0.096354  0.097917   0.097917  1443120000   
1  1997-05-16  0.098438  0.098958  0.085417  0.086458   0.086458   294000000   
2  1997-05-19  0.088021  0.088542  0.081250  0.085417   0.085417   122136000   
3  1997-05-20  0.086458  0.087500  0.081771  0.081771   0.081771   109344000   
4  1997-05-21  0.081771  0.082292  0.068750  0.071354   0.071354   377064000   

   Dividends  Stock Splits stock_symbol  
0        0.0           0.0         AMZN  
1        0.0           0.0         AMZN  
2        0.0           0.0         AMZN  
3        0.0           0.0         AMZN  
4        0.0           0.0         AMZN  


In [14]:
#Sentment data
stock_data=corr_analyser.merge_csv_files(folder_path)
news_data=pd.read_csv('../data/raw_analyst_ratings/raw_analyst_ratings.csv')

# drop data
newData = news_data.drop(['Unnamed: 0'], axis=1)

In [15]:
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,stock_symbol
0,1997-05-15,0.121875,0.125000,0.096354,0.097917,0.097917,1443120000,0.0,0.0,AMZN
1,1997-05-16,0.098438,0.098958,0.085417,0.086458,0.086458,294000000,0.0,0.0,AMZN
2,1997-05-19,0.088021,0.088542,0.081250,0.085417,0.085417,122136000,0.0,0.0,AMZN
3,1997-05-20,0.086458,0.087500,0.081771,0.081771,0.081771,109344000,0.0,0.0,AMZN
4,1997-05-21,0.081771,0.082292,0.068750,0.071354,0.071354,377064000,0.0,0.0,AMZN
...,...,...,...,...,...,...,...,...,...,...
45423,2024-07-24,175.389999,177.949997,173.570007,174.369995,174.369995,31250700,0.0,0.0,GOOG
45424,2024-07-25,174.250000,175.199997,169.050003,169.160004,169.160004,28967900,0.0,0.0,GOOG
45425,2024-07-26,168.770004,169.839996,165.865005,168.679993,168.679993,25150100,0.0,0.0,GOOG
45426,2024-07-29,170.500000,172.160004,169.720001,171.130005,171.130005,13768900,0.0,0.0,GOOG


In [16]:
newData


,headline,url,publisher,date,stock
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A
...,...,...,...,...,...
1407323,Top Narrow Based Indexes For August 29,https://www.benzinga.com/news/11/08/1888782/to...,Monica Gerson,2011-08-29 00:00:00,ZX
1407324,Recap: Wednesday's Top Percentage Gainers and ...,https://www.benzinga.com/news/earnings/11/06/1...,Benjamin Lee,2011-06-22 00:00:00,ZX
1407325,UPDATE: Oppenheimer Color on China Zenix Auto ...,https://www.benzinga.com/analyst-ratings/analy...,BenzingaStaffL,2011-06-21 00:00:00,ZX
1407326,Oppenheimer Initiates China Zenix At Outperfor...,https://www.benzinga.com/analyst-ratings/price...,Joe Young,2011-06-21 00:00:00,ZX


##### Data preparation

In [18]:
# Convert the 'date' column to datetime format and UTC-4 timezone
stock_data['Date'] = pd.to_datetime(stock_data['Date'], format='ISO8601', utc=True)
newData['date']=pd.to_datetime(newData['date'], format='ISO8601', utc=True)
print("Minimum date from news dataset:  ",newData.date.min())
print("Maximum date from news dataset:  ",newData.date.max())
print("Minimum date from Stock dataset:  ",stock_data.Date.min())
print("Max date from Stock dataset:  ",stock_data.Date.max())

Minimum date from news dataset:   2009-02-14 04:00:00+00:00
Maximum date from news dataset:   2020-06-11 21:12:35+00:00
Minimum date from Stock dataset:   1980-12-12 00:00:00+00:00
Max date from Stock dataset:   2024-07-30 00:00:00+00:00
